# Projet: algorithme et programmation

## Théme: Analyse des offres d'emploi en Côte d'Ivoire : cas des offres publiées sur le site emploi.ci.

### Scrapping des donnees sur le site Job COTE D'IVOIRE

In [30]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL de base
BASE_URL = "https://www.emploi.ci/recherche-jobs-cote-ivoire"

# Fonction pour extraire les informations d'une page
def extract_jobs(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur lors du chargement de la page {url}")
        return []
    
    soup = BeautifulSoup(response.content, 'html.parser')
    job_cards = soup.find_all('div', class_='card-job-detail')
    
    jobs = []
    for job in job_cards:
        title = job.find('h3').get_text(strip=True) if job.find('h3') else "Non spécifié"
        company = job.find('a', class_='card-job-company').get_text(strip=True) if job.find('a', class_='card-job-company') else "Non spécifié"
        description = job.find('div', class_='card-job-description').p.get_text(strip=True) if job.find('div', class_='card-job-description') else "Non spécifié"
        
        education_level = "Non spécifié"
        education_li = job.find_all('li')
        for li in education_li:
            if 'Niveau d´études requis' in li.get_text(strip=True):
                education_level = li.find('strong').get_text(strip=True) if li.find('strong') else "Non spécifié"
                break
        
        experience_level = "Non spécifié"
        for li in education_li:
            if 'Niveau d\'expérience' in li.get_text(strip=True):
                experience_level = li.find('strong').get_text(strip=True) if li.find('strong') else "Non spécifié"
                break
        
        contract = "Non spécifié"
        for li in education_li:
            if 'Contrat proposé' in li.get_text(strip=True):
                contract = li.find('strong').get_text(strip=True) if li.find('strong') else "Non spécifié"
                break
        
        region = "Non spécifié"
        for li in education_li:
            if 'Région de' in li.get_text(strip=True):
                region = li.find('strong').get_text(strip=True) if li.find('strong') else "Non spécifié"
                break
        
        skills = "Non spécifié"
        for li in education_li:
            if 'Compétences clés' in li.get_text(strip=True):
                skills = li.find('strong').get_text(strip=True) if li.find('strong') else "Non spécifié"
                break

        date = job.find('time')['datetime'] if job.find('time') else "Non spécifié"
        
        jobs.append({
            'Titre': title,
            'Entreprise': company,
            'Description': description,
            'Niveau d\'études': education_level,
            'Expérience': experience_level,
            'Contrat': contract,
            'Région': region,
            'Compétences': skills,
            'Date': date
        })
    return jobs

# Fonction pour scraper toutes les pages
def scrape_all_jobs(base_url):
    all_jobs = []  # Liste pour stocker toutes les offres

    # Scraping de la première page (non paginée)
    print("Scraping de la première page (non paginée)...")
    first_page_jobs = extract_jobs(base_url)
    all_jobs.extend(first_page_jobs)

    # Scraping des pages suivantes (paginées)
    page = 1  # correspond à la 2nd page
    while True:
        print(f"Scraping page {page}...")
        paginated_url = f"{base_url}?page={page}"
        jobs = extract_jobs(paginated_url)
        if not jobs:  # Arrêt si aucune offre n'est trouvée
            print("Aucune nouvelle offre, arrêt du scraping.")
            break
        all_jobs.extend(jobs)
        page += 1

    return all_jobs

# Exécution du scraping
print("Début du scraping...")
all_jobs = scrape_all_jobs(BASE_URL)


# Sauvegarder dans un fichier CSV
df = pd.DataFrame(all_jobs)
df.to_csv(r"C:\Users\HP\OneDrive\Desktop\Cours\ISE\ISE ECO\Algo et programmation\projet\data", index=False, encoding='utf-8-sig')

print("Scraping terminé. Données sauvegardées dans data.csv")

Scraping la première page...
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping terminé. Données sauvegardées dans data.csv


### Traitement des donnees

In [31]:
import pandas as pd
file_path = r"C:\Users\HP\OneDrive\Desktop\Cours\ISE\ISE ECO\Algo et programmation\projet\data"  # Remplacez par le chemin vers votre fichier
data = pd.read_csv(file_path)
data.head()

,Titre,Entreprise,Description,Niveau d'études,Expérience,Contrat,Région,Compétences,Date
0,Chargé de Recouvrement H/F,TECTRA CI,Nous sommes à la recherche d'un Chargé de Rec...,Bac+5 et plus,Expérience entre 2 ans et 5 ans,CDI & CDD,Abidjan,Banque - Comptabilité - Finance - Gestion - Re...,2024-12-22
1,Coordinateur Superviseurs de Site - Abidjan,TECTRA CI,Nous sommes à la recherche des Coordinateurs ...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,CDI,Abidjan - Aboisso - Adzopé - Dabou & Divo,Accompagnement - Gestion d'Équipe - Gestion de...,2024-12-22
2,Chargés de Clientèle Francophones - Treichville,MAJOREL,Nous sommes à la recherche des Chargés de Cli...,Bac,"Etudiant, jeune diplômé & Débutant < 2 ans",Stage,Abidjan,Accompagnement - Hôtellerie - Restauration - T...,2024-12-20
3,Responsable Commercial Pays - Abidjan,AFRICAWORK,Nous sommes à la recherche d'un Responsable C...,Bac+4 & Bac+5 et plus,Expérience entre 5 ans et 10 ans & Expérience ...,CDI,Abidjan,Actions Commerciales - Négociation - Stratégie...,2024-12-20
4,Commercial - Abidjan,LOGODOUMAN,Nous recherchons notre futur(e) Commercial(e)...,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans,CDD,Abidjan,Non spécifié,2024-12-19


In [32]:
# description de la base de donnée
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Titre            126 non-null    object
 1   Entreprise       126 non-null    object
 2   Description      126 non-null    object
 3   Niveau d'études  126 non-null    object
 4   Expérience       126 non-null    object
 5   Contrat          126 non-null    object
 6   Région           126 non-null    object
 7   Compétences      126 non-null    object
 8   Date             126 non-null    object
dtypes: object(9)
memory usage: 9.0+ KB


In [33]:
# verification de donnees manquantes
data.isna().sum()

Titre              0
Entreprise         0
Description        0
Niveau d'études    0
Expérience         0
Contrat            0
Région             0
Compétences        0
Date               0
dtype: int64

In [34]:
# Ajouter une colonne 'ID'
data['ID'] = range(1, len(data) + 1)

# Réorganiser les colonnes pour que 'ID' soit en premier
data = data[['ID'] + [col for col in data.columns if col != 'ID']]

data.head()

,ID,Titre,Entreprise,Description,Niveau d'études,Expérience,Contrat,Région,Compétences,Date
0,1,Chargé de Recouvrement H/F,TECTRA CI,Nous sommes à la recherche d'un Chargé de Rec...,Bac+5 et plus,Expérience entre 2 ans et 5 ans,CDI & CDD,Abidjan,Banque - Comptabilité - Finance - Gestion - Re...,2024-12-22
1,2,Coordinateur Superviseurs de Site - Abidjan,TECTRA CI,Nous sommes à la recherche des Coordinateurs ...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,CDI,Abidjan - Aboisso - Adzopé - Dabou & Divo,Accompagnement - Gestion d'Équipe - Gestion de...,2024-12-22
2,3,Chargés de Clientèle Francophones - Treichville,MAJOREL,Nous sommes à la recherche des Chargés de Cli...,Bac,"Etudiant, jeune diplômé & Débutant < 2 ans",Stage,Abidjan,Accompagnement - Hôtellerie - Restauration - T...,2024-12-20
3,4,Responsable Commercial Pays - Abidjan,AFRICAWORK,Nous sommes à la recherche d'un Responsable C...,Bac+4 & Bac+5 et plus,Expérience entre 5 ans et 10 ans & Expérience ...,CDI,Abidjan,Actions Commerciales - Négociation - Stratégie...,2024-12-20
4,5,Commercial - Abidjan,LOGODOUMAN,Nous recherchons notre futur(e) Commercial(e)...,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans,CDD,Abidjan,Non spécifié,2024-12-19


In [35]:
# retirer les noms des villes dans les titres des postes
import re
# Fonction pour supprimer les lieux de poste à la fin du titre
def supprimer_lieux_du_titre(titre):
    # Enlève tout ce qui suit un tiret et qui est un lieu (mot à la fin)
    titre = re.sub(r'\s*-\s*\w+$', '', titre)
    return titre.strip()

# Appliquer la fonction sur la colonne 'titre_poste' du DataFrame
data["Titre de l'offre"] = data['Titre'].apply(supprimer_lieux_du_titre)

# Afficher les premiers résultats pour vérifier
data.head(10)

,ID,Titre,Entreprise,Description,Niveau d'études,Expérience,Contrat,Région,Compétences,Date,Titre de l'offre
0,1,Chargé de Recouvrement H/F,TECTRA CI,Nous sommes à la recherche d'un Chargé de Rec...,Bac+5 et plus,Expérience entre 2 ans et 5 ans,CDI & CDD,Abidjan,Banque - Comptabilité - Finance - Gestion - Re...,2024-12-22,Chargé de Recouvrement H/F
1,2,Coordinateur Superviseurs de Site - Abidjan,TECTRA CI,Nous sommes à la recherche des Coordinateurs ...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,CDI,Abidjan - Aboisso - Adzopé - Dabou & Divo,Accompagnement - Gestion d'Équipe - Gestion de...,2024-12-22,Coordinateur Superviseurs de Site
2,3,Chargés de Clientèle Francophones - Treichville,MAJOREL,Nous sommes à la recherche des Chargés de Cli...,Bac,"Etudiant, jeune diplômé & Débutant < 2 ans",Stage,Abidjan,Accompagnement - Hôtellerie - Restauration - T...,2024-12-20,Chargés de Clientèle Francophones
3,4,Responsable Commercial Pays - Abidjan,AFRICAWORK,Nous sommes à la recherche d'un Responsable C...,Bac+4 & Bac+5 et plus,Expérience entre 5 ans et 10 ans & Expérience ...,CDI,Abidjan,Actions Commerciales - Négociation - Stratégie...,2024-12-20,Responsable Commercial Pays
4,5,Commercial - Abidjan,LOGODOUMAN,Nous recherchons notre futur(e) Commercial(e)...,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans,CDD,Abidjan,Non spécifié,2024-12-19,Commercial
5,6,Chargé(e) de Relations Publiques et Partenaria...,AFROBEAT INTERNATIONAL,Afrobeat International recherche un(e) Chargé...,"Bac, Bac+1, Bac+2, Bac+3 & Bac+4","Etudiant, jeune diplômé et plus",CDD,Abidjan,Assistanat - Communication - Relations Publiqu...,2024-12-19,Chargé(e) de Relations Publiques et Partenariats
6,7,Professeur ou Tuteur de Maths ou Physique-Chim...,COURS-EN-VISIO,"Nous sommes une startup française, dont la pla...","Bac+2, Bac+3, Bac+4 & Bac+5 et plus","Etudiant, jeune diplômé et plus",Freelance & Temps partiel,Abidjan,Accompagnement,2024-12-19,Professeur ou Tuteur de Maths ou Physique-Chimie
7,8,COMMERCIAUX DE VEHICULES POIDS LOURDS ET MACHI...,ZONDACI SARL,En tant que Commercial(e) de véhicules poids l...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans et plus,CDD,Abidjan - Bouaké - Korhogo & Yamoussoukro,Communication - Communication de Produit - Mar...,2024-12-19,COMMERCIAUX DE VEHICULES POIDS LOURDS ET MACHI...
8,9,Community Manager - Abidjan,SMYRNE GROUPE CI,Nous sommes à la recherche d'un Community Man...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans et plus,CDI & CDD,Abidjan,Maîtrise Technique - Vente,2024-12-19,Community Manager
9,10,Référenceur(se) SEO Google / Webmaster - Télé...,OPTIMUM INTERIM,Nous sommes à la recherche des Référenceurs S...,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,Freelance,International,Référencement Naturel - Référencement SEO - Ré...,2024-12-18,Référenceur(se) SEO Google / Webmaster


In [36]:
# Extraire le niveau minimum d'études requis
data['Niveau minimum'] = data["Niveau d'études"].str.extract(r'(Bac\+\d+)')

# Extraire le nombre minimum d'années d'expérience ("Débutant" si aucune précision)
data['Expérience minimum'] = data['Expérience'].str.extract(r'(\d+)').fillna('Débutant')

# Séparer les types de contrats en colonnes binaires
data['CDI'] = data['Contrat'].str.contains('CDI', na=False).astype(int)
data['CDD'] = data['Contrat'].str.contains('CDD', na=False).astype(int)
data['Stage'] = data['Contrat'].str.contains('Stage', na=False).astype(int)

# separer les regions:

# Nettoyer et normaliser la région
data['Région 1'] = data['Région'].str.strip()

### Preparation des colonnees texte pour faire un nuage des mots:

In [37]:
#!pip install pandas nltk

In [38]:
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

In [39]:
# Télécharger les ressources nécessaires
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [40]:
# Liste des stopwords en français
stop_words = set(stopwords.words('french'))

# Initialiser le lemmatizer
lemmatizer = WordNetLemmatizer()

In [41]:
# Fonction de nettoyage des textes
def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return ''
    # Convertir en minuscule
    texte = texte.lower()
    # Supprimer la ponctuation
    texte = texte.translate(str.maketrans('', '', string.punctuation))
    # Diviser en mots
    mots = texte.split()
    # Supprimer les stopwords et lemmatiser
    mots = [lemmatizer.lemmatize(mot) for mot in mots if mot not in stop_words]
    return ' '.join(mots)

In [42]:
df = data.copy()
# Nettoyer la colonne Competences
df['Competences_nettoye'] = df['Compétences'].apply(nettoyer_texte)

# Nettoyer la colonne Region
df['Region_nettoye'] = df['Région'].apply(nettoyer_texte)

In [43]:
df.head()

,ID,Titre,Entreprise,Description,Niveau d'études,Expérience,Contrat,Région,Compétences,Date,Titre de l'offre,Niveau minimum,Expérience minimum,CDI,CDD,Stage,Région 1,Competences_nettoye,Region_nettoye
0,1,Chargé de Recouvrement H/F,TECTRA CI,Nous sommes à la recherche d'un Chargé de Rec...,Bac+5 et plus,Expérience entre 2 ans et 5 ans,CDI & CDD,Abidjan,Banque - Comptabilité - Finance - Gestion - Re...,2024-12-22,Chargé de Recouvrement H/F,Bac+5,2,1,1,0,Abidjan,banque comptabilité finance gestion recouvrement,abidjan
1,2,Coordinateur Superviseurs de Site - Abidjan,TECTRA CI,Nous sommes à la recherche des Coordinateurs ...,"Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Expérience entre 2 ans et 5 ans et plus,CDI,Abidjan - Aboisso - Adzopé - Dabou & Divo,Accompagnement - Gestion d'Équipe - Gestion de...,2024-12-22,Coordinateur Superviseurs de Site,Bac+2,2,1,0,0,Abidjan - Aboisso - Adzopé - Dabou & Divo,accompagnement gestion déquipe gestion ressour...,abidjan aboisso adzopé dabou divo
2,3,Chargés de Clientèle Francophones - Treichville,MAJOREL,Nous sommes à la recherche des Chargés de Cli...,Bac,"Etudiant, jeune diplômé & Débutant < 2 ans",Stage,Abidjan,Accompagnement - Hôtellerie - Restauration - T...,2024-12-20,Chargés de Clientèle Francophones,NaN,2,0,0,1,Abidjan,accompagnement hôtellerie restauration tourism...,abidjan
3,4,Responsable Commercial Pays - Abidjan,AFRICAWORK,Nous sommes à la recherche d'un Responsable C...,Bac+4 & Bac+5 et plus,Expérience entre 5 ans et 10 ans & Expérience ...,CDI,Abidjan,Actions Commerciales - Négociation - Stratégie...,2024-12-20,Responsable Commercial Pays,Bac+4,5,1,0,0,Abidjan,action commerciales négociation stratégie comm...,abidjan
4,5,Commercial - Abidjan,LOGODOUMAN,Nous recherchons notre futur(e) Commercial(e)...,"Bac, Bac+1, Bac+2, Bac+3, Bac+4 & Bac+5 et plus",Débutant < 2 ans,CDD,Abidjan,Non spécifié,2024-12-19,Commercial,Bac+1,2,0,1,0,Abidjan,non spécifié,abidjan


In [5]:
print(data['Date'].min(),'-',data['Date'].max())

2024-11-08 - 2024-12-22


In [44]:
# Sauvegarder dans un fichier CSV
df.to_csv(r"C:\Users\HP\OneDrive\Desktop\Cours\ISE\ISE ECO\Algo et programmation\projet\data_nettoyé", index=False, encoding='utf-8-sig')